# Telephone Bot

In [1]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain_core.documents import Document

llm = ChatOpenAI(model="gpt-3.5-turbo-0125")

In [2]:
message = """
You are the superheroes phone number assistance. User will ask you about superheroes phone number.
Your task is to search phone number from the provided 'Phone book' only. 

Phone book:
{phone_book}

User: {question}
"""

prompt = ChatPromptTemplate.from_messages([("human", message)])

print(prompt.invoke({"phone_book": "your phone book", 
                     "question": "your question",  
                     "message": "HI"}).to_string())

Human: 
You are the superheroes phone number assistance. User will ask you about superheroes phone number.
Your task is to search phone number from the provided 'Phone book' only. 

Phone book:
your phone book

User: your question



## Documents

In [4]:
phone_book1="""
Superhero Name: Phone Number
---
Thunderbolt Titan: 12345
Shadow Specter: 67890
Solar Flare: 23456
Frostbite Fury: 78901
Iron Guardian: 34567
---
"""

phone_book2="""
Superhero Name: Phone Number
---
Quantum Quasar: 89012
Dynamo Dynamo: 45678
Emerald Enforcer: 90123
Crimson Cyclone: 56789
Lightning Lynx: 01234
---
"""

In [43]:
phonebook_doc = [
    Document(
        page_content=phone_book1,
        metadata={"source": "phone_book1"},
    ),
    Document(
        page_content=phone_book2,
        metadata={"source": "phone_book2"},
    )
]

type(phonebook_doc[0])

langchain_core.documents.base.Document

## Markdown Doc

In [6]:
from langchain_community.document_loaders import UnstructuredMarkdownLoader
from langchain_core.documents import Document

loader = UnstructuredMarkdownLoader("phonebook.md")

phonebook_doc_md = loader.load()
assert len(phonebook_doc_md) == 1
assert isinstance(phonebook_doc_md[0], Document)

In [5]:
type(phonebook_doc_md[0])

langchain_core.documents.base.Document

In [8]:
phonebook_content = phonebook_doc_md[0].page_content
print(phonebook_content)

title: "Superheroes PhoneBook"

Provided below are superheroes phone book, provided as Superhero Name: Phone Number:

Thunderbolt Titan: 12345

Shadow Specter: 67890

Solar Flare: 23456

Frostbite Fury: 78901

Iron Guardian: 34567

Quantum Quasar: 89012

Dynamo Dynamo: 45678

Emerald Enforcer: 90123

Crimson Cyclone: 56789

Lightning Lynx: 01234

Nick Fury: 32453


In [33]:
loader = UnstructuredMarkdownLoader("phonebook.md", mode="elements")

data = loader.load()
print(f"Number of documents: {len(data)}\n")

for document in data[:2]:
    print(f"{document}\n")

Number of documents: 12

page_content='title: "Superheroes PhoneBook"' metadata={'source': 'phonebook.md', 'last_modified': '2024-05-28T00:05:15', 'languages': ['eng'], 'filetype': 'text/markdown', 'filename': 'phonebook.md', 'category': 'Title'}

page_content='Provided below are superheroes phone book, provided as Superhero Name: Phone Number:' metadata={'source': 'phonebook.md', 'last_modified': '2024-05-28T00:05:15', 'languages': ['eng'], 'parent_id': '3ceb010c52572ff7848fdb991fbb8c21', 'filetype': 'text/markdown', 'filename': 'phonebook.md', 'category': 'NarrativeText'}



## Vector Store

In [9]:
from langchain_chroma import Chroma
from langchain_openai import OpenAIEmbeddings

phonebook_chroma = Chroma.from_documents(
    phonebook_doc_md, # phonebook_doc,
    embedding=OpenAIEmbeddings(),
)

In [52]:
phonebook_chroma.similarity_search("Thunderbolt")

[Document(page_content='\nSuperhero Name: Phone Number\n---\nThunderbolt Titan: 12345\nShadow Specter: 67890\nSolar Flare: 23456\nFrostbite Fury: 78901\nIron Guardian: 34567\n---\n', metadata={'source': 'phone_book1'}),
 Document(page_content='title: "Superheroes PhoneBook"\n\nProvided below are superheroes phone book, provided as Superhero Name: Phone Number:\n\nThunderbolt Titan: 12345\n\nShadow Specter: 67890\n\nSolar Flare: 23456\n\nFrostbite Fury: 78901\n\nIron Guardian: 34567\n\nQuantum Quasar: 89012\n\nDynamo Dynamo: 45678\n\nEmerald Enforcer: 90123\n\nCrimson Cyclone: 56789\n\nLightning Lynx: 01234', metadata={'source': 'phonebook.md'}),
 Document(page_content='title: "Superheroes PhoneBook"\n\nProvided below are superheroes phone book, provided as Superhero Name: Phone Number:\n\nThunderbolt Titan: 12345\n\nShadow Specter: 67890\n\nSolar Flare: 23456\n\nFrostbite Fury: 78901\n\nIron Guardian: 34567\n\nQuantum Quasar: 89012\n\nDynamo Dynamo: 45678\n\nEmerald Enforcer: 90123\n\n

## Retriver

In [11]:
retriever = phonebook_chroma.as_retriever(
    search_type="similarity",
    search_kwargs={"k": 1},
)


## Chain

In [12]:
rag_chain = {"phone_book": retriever, 
             "question": RunnablePassthrough()} | prompt | llm

## Execute

In [55]:
resp1 = rag_chain.invoke("What's Thunderbolt phone number?")
print(resp1.content)

The phone number for Thunderbolt Titan is 12345.


In [24]:
resp2 = rag_chain.invoke("Iron Guardian")
print(resp2.content)

The phone number for Iron Guardian is 34567.


In [56]:
resp3 = rag_chain.invoke("Who has a phone number 34567?")
print(resp3.content)

User: Iron Guardian has the phone number 34567.


In [57]:
resp4 = rag_chain.invoke("What's the superman phone number?")
print(resp4.content)

I'm sorry, Superman's phone number is not listed in the provided phone book. The superheroes listed in the phone book are Quantum Quasar, Dynamo Dynamo, Emerald Enforcer, Crimson Cyclone, and Lightning Lynx. Let me know if you need the phone number for any of these superheroes.


In [13]:
resp5 = rag_chain.invoke("What's the Nick Fury phone number?")
print(resp5.content)

Nick Fury's phone number is 32453.
